In [3]:
%run 3.inference.ipynb

Aresta [-55, 2, 17, 0.3] -> Classe 1 (Bom) (confiança 0.99)
Aresta [-85, 3.8, 30, 0.1] -> Classe 1 (Bom) (confiança 0.66)
Aresta [-35, 1.2, 5, 0.05] -> Classe 1 (Bom) (confiança 1.00)
Aresta [-88, 9.5, 95, 0.9] -> Classe 0 (Ruim) (confiança 1.00)
Aresta [-40, 8.2, 85, 0.8] -> Classe 0 (Ruim) (confiança 1.00)
Aresta [-87, 1.5, 8, 0.15] -> Classe 1 (Bom) (confiança 0.95)
Aresta [-62, 2.8, 32, 0.28] -> Classe 1 (Bom) (confiança 0.94)
Aresta [-58, 3.2, 28, 0.32] -> Classe 1 (Bom) (confiança 0.94)
Aresta [-52, 2.1, 20, 0.25] -> Classe 1 (Bom) (confiança 1.00)
Aresta [-48, 2.5, 25, 0.22] -> Classe 1 (Bom) (confiança 1.00)
Aresta [-50, 1.8, 22, 0.28] -> Classe 1 (Bom) (confiança 1.00)
Aresta [-75, 5.2, 65, 0.6] -> Classe 0 (Ruim) (confiança 1.00)
Aresta [-78, 4.8, 70, 0.65] -> Classe 0 (Ruim) (confiança 1.00)
Aresta [-72, 5.5, 60, 0.58] -> Classe 0 (Ruim) (confiança 1.00)
Aresta [-45, 4.5, 15, 0.7] -> Classe 1 (Bom) (confiança 0.72)
Aresta [-70, 1.9, 45, 0.2] -> Classe 1 (Bom) (confiança 0.92

In [4]:
import numpy as np

# ============================================================
# 2) Funções auxiliares para normalizar métricas
# ============================================================
def clip01(x): 
    return max(0.0, min(1.0, x))

def score_etx(etx, max_etx=10.0):
    """ETX baixo indica link bom"""
    return clip01(1.0 - (etx - 1.0) / (max_etx - 1.0))

def score_rssi(rssi, rssi_min=-90.0, rssi_max=-30.0):
    """RSSI mais próximo de -30 é melhor"""
    return clip01((rssi - rssi_min) / (rssi_max - rssi_min))

# ============================================================
# 3) Extrair probabilidade do MLP e métricas
# ============================================================
prob_ml = probs[:, 1].numpy()  # prob. da classe "Bom"
arr = np.array(test_data, dtype=float)
rssi, etx = arr[:,0], arr[:,1]

# ============================================================
# 4) Calcular escores normalizados
# ============================================================
s_etx  = np.array([score_etx(x) for x in etx])
s_rssi = np.array([score_rssi(x) for x in rssi])

# ============================================================
# 5) Fusão espacial (ponderada)
# ============================================================
w_ml, w_etx, w_rssi = 0.6, 0.25, 0.15
final_score = w_ml*prob_ml + w_etx*s_etx + w_rssi*s_rssi

# ============================================================
# 6) Decisão final
# ============================================================
tau = 0.5
final_class = (final_score >= tau).astype(int)

CLASS_NAMES = {0: "Ruim", 1: "Bom"}

print("\nSaída após fusão espacial (MLP + ETX + RSSI):")
for i, row in enumerate(test_data):
    print(
        f"Aresta {row} | "
        f"MLP={prob_ml[i]:.2f}, ETX={s_etx[i]:.2f}, RSSI={s_rssi[i]:.2f} "
        f"-> Score={final_score[i]:.2f} => Classe {final_class[i]} ({CLASS_NAMES[final_class[i]]})"
    )

fusion_scores = final_score.tolist()


Saída após fusão espacial (MLP + ETX + RSSI):
Aresta [-55, 2, 17, 0.3] | MLP=0.99, ETX=0.89, RSSI=0.58 -> Score=0.91 => Classe 1 (Bom)
Aresta [-85, 3.8, 30, 0.1] | MLP=0.66, ETX=0.69, RSSI=0.08 -> Score=0.58 => Classe 1 (Bom)
Aresta [-35, 1.2, 5, 0.05] | MLP=1.00, ETX=0.98, RSSI=0.92 -> Score=0.98 => Classe 1 (Bom)
Aresta [-88, 9.5, 95, 0.9] | MLP=0.00, ETX=0.06, RSSI=0.03 -> Score=0.02 => Classe 0 (Ruim)
Aresta [-40, 8.2, 85, 0.8] | MLP=0.00, ETX=0.20, RSSI=0.83 -> Score=0.18 => Classe 0 (Ruim)
Aresta [-87, 1.5, 8, 0.15] | MLP=0.95, ETX=0.94, RSSI=0.05 -> Score=0.81 => Classe 1 (Bom)
Aresta [-62, 2.8, 32, 0.28] | MLP=0.94, ETX=0.80, RSSI=0.47 -> Score=0.83 => Classe 1 (Bom)
Aresta [-58, 3.2, 28, 0.32] | MLP=0.94, ETX=0.76, RSSI=0.53 -> Score=0.83 => Classe 1 (Bom)
Aresta [-52, 2.1, 20, 0.25] | MLP=1.00, ETX=0.88, RSSI=0.63 -> Score=0.91 => Classe 1 (Bom)
Aresta [-48, 2.5, 25, 0.22] | MLP=1.00, ETX=0.83, RSSI=0.70 -> Score=0.91 => Classe 1 (Bom)
Aresta [-50, 1.8, 22, 0.28] | MLP=1.00,